#   LangChain Tool 활용

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


---

## **내장 도구 (Built-in Tool)**

- **내장 도구**는 시스템에 **기본적으로 포함**된 사전 정의된 도구들의 집합

- 파일 처리, 웹 검색, 코드 실행 등 **자주 사용되는 기본 기능**들을 즉시 활용 가능

- 별도의 구현 없이 **바로 사용**할 수 있어 개발 시간과 노력을 절약 가능

---

### **웹 검색(Tavily Search API)**

- **Tavily**는 AI 에이전트(LLM)를 위해 특별히 설계된 검색 엔진

- 검색 결과로 **제목, URL, 컨텐츠, 답변** 등 상세 정보 제공

- 개발자는 **월 1,000회** 무료 검색 쿼터 사용 가능

- **설치**: `pip install tavily-python` 또는 `uv add tavily-python`

- **인증키**: `TAVILY_API_KEY`

`(1) 도구 정의`

In [7]:
from langchain_community.tools import TavilySearchResults

search_web = TavilySearchResults(
    max_results=5,  # 반환할 결과의 수
    search_depth="advanced",  # 검색 깊이: basic 또는 advanced
    include_answer=True,  # 결과에 직접적인 답변 포함
    include_raw_content=True,  # 페이지의 원시 콘텐츠 포함
    include_images=True,  # 결과에 이미지 포함
    # include_domains=[...],  # 특정 도메인으로 검색 제한
    # exclude_domains=[...],  # 특정 도메인 제외
    # name="...",  # 기본 도구 이름 덮어쓰기
    # description="...",  # 기본 도구 설명 덮어쓰기
    # args_schema=...,  # 기본 args_schema 덮어쓰기
)

`(2) 도구 직접 호출`
- 자연어 쿼리를 도구에 전달

In [8]:
result = search_web.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
pprint(result)

("HTTPError('401 Client Error: Unauthorized for url: "
 "https://api.tavily.com/search')")


`(3) ToolCall을 사용한 호출`
- 모델에서 생성된 `ToolCall`을 사용하여 도구를 호출

In [9]:
from langchain_openai import ChatOpenAI

# 모델 생성
model = ChatOpenAI(model="gpt-4.1-mini")

# 모델에 도구 등록 
model_with_tools = model.bind_tools([search_web])

# 사용자 쿼리를 입력하여 ToolCall 생성
response = model_with_tools.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")

# ToolCall 출력 
pprint(response.tool_calls)

[{'args': {'query': '한국 시장 거래 ETF 종목 수'},
  'id': 'call_kmtOrVCUjxogDknB2QD1yQUm',
  'name': 'tavily_search_results_json',
  'type': 'tool_call'}]


In [10]:
# ToolCall을 사용하여 도구 실행
model_generated_tool_call = response.tool_calls[0]
tool_msg = search_web.invoke(model_generated_tool_call)

# 도구 실행 결과 출력
pprint(tool_msg.content)

("HTTPError('401 Client Error: Unauthorized for url: "
 "https://api.tavily.com/search')")


In [ ]:
# `artifact` 속성 출력
tool_msg.artifact

In [ ]:
pprint(tool_msg.artifact)

`(4) LLM 체인과 연계`
- Tavily 도구를 언어 모델에 바인딩하고 사용자 입력을 처리하는 체인을 생성

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import ToolMessage

import datetime

# 언어 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 프롬프트 템플릿 정의
today = datetime.datetime.today().strftime("%Y-%m-%d")
prompt = ChatPromptTemplate(
    [
        ("system", f"당신은 도움이 되는 어시스턴트입니다. 오늘 날짜는 {today}입니다."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# Tavily 도구를 모델에 바인딩
llm_with_tools = llm.bind_tools([search_web])

# 체인 생성
llm_chain = prompt | llm_with_tools

@chain  # 함수를 체인으로 사용하기 위해 데코레이터 추가
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    
    # 도구 호출 결과를 처리
    tool_msgs = []
    artifacts = []  # artifact를 저장할 리스트
    for tool_call in ai_msg.tool_calls:
        tool_result = search_web.invoke(tool_call)
        tool_msgs.append(ToolMessage(content=str(tool_result.content), tool_call_id=tool_call["id"]))
        artifacts.append(tool_result.artifact)  # artifact 저장
    
    # 최종 결과 반환
    final_response = llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)
    
    return final_response, artifacts

# 체인 호출
response, artifacts = tool_chain.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
print("===== 최종 응답 =====")
print(response.content)

In [ ]:
# artifact를 별도로 출력
print("===== Artifact 정보 =====")
for artifact in artifacts:
    print(json.dumps(artifact, indent=2, ensure_ascii=False))

---

##  **사용자 정의 도구 (Custom Tool)**


- **사용자 정의 도구**는 개발자가 직접 설계하고 구현하는 **맞춤형 함수나 도구**를 의미

- LLM이 호출할 수 있는 **고유한 기능**을 정의하여 특정 작업에 최적화된 도구 생성 가능

- 개발자는 도구의 **입력값, 출력값, 기능**을 자유롭게 정의하여 유연한 확장성 확보

---

### 1. **외부 API 연동** 

- LangChain에서는 **사용자 정의 도구**를 통해 외부 API와의 연동이 가능

- **Tool** 클래스를 상속받아 필요한 기능을 구현하며, `name`과 `description`을 필수로 정의

- 도구는 단일 기능을 수행하는 **함수 형태**로 구현되며, 입력과 출력이 명확하게 설정함

`(1) Yahoo Finance API` 

In [11]:
#  yfinance 설치 : pip install yfinance 또는 uv add yfinance
import yfinance as yf

dat = yf.Ticker("MSFT")

In [12]:
# 기업정보
dat.info

{'address1': 'One Microsoft Way',
 'city': 'Redmond',
 'state': 'WA',
 'zip': '98052-6399',
 'country': 'United States',
 'phone': '425 882 8080',
 'website': 'https://www.microsoft.com',
 'industry': 'Software - Infrastructure',
 'industryKey': 'software-infrastructure',
 'industryDisp': 'Software - Infrastructure',
 'sector': 'Technology',
 'sectorKey': 'technology',
 'sectorDisp': 'Technology',
 'longBusinessSummary': 'Microsoft Corporation develops and supports software, services, devices and solutions worldwide. The Productivity and Business Processes segment offers office, exchange, SharePoint, Microsoft Teams, office 365 Security and Compliance, Microsoft viva, and Microsoft 365 copilot; and office consumer services, such as Microsoft 365 consumer subscriptions, Office licensed on-premises, and other office services. This segment also provides LinkedIn; and dynamics business solutions, including Dynamics 365, a set of intelligent, cloud-based applications across ERP, CRM, power 

In [13]:
# 주요 일정
dat.calendar

{'Dividend Date': datetime.date(2025, 9, 11),
 'Ex-Dividend Date': datetime.date(2025, 8, 21),
 'Earnings Date': [datetime.date(2025, 7, 30)],
 'Earnings High': 3.57,
 'Earnings Low': 3.29784,
 'Earnings Average': 3.37462,
 'Revenue High': 74464996940,
 'Revenue Low': 72570000000,
 'Revenue Average': 73792496670}

In [15]:
# 2022년 1월 3일 ~ 4일의 데이터를 판다스 데이터프레임으로 출력 
result = dat.history(start="2025-01-03", end="2025-05-05") 
result

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2025-01-03 00:00:00-05:00,419.467282,422.405996,417.933202,421.728607,16662900,0.0,0.0
2025-01-06 00:00:00-05:00,426.360784,432.656586,423.850447,426.211365,20573600,0.0,0.0
2025-01-07 00:00:00-05:00,427.356962,429.000637,419.188356,420.752350,18139100,0.0,0.0
2025-01-08 00:00:00-05:00,421.838197,425.334764,419.925568,422.933990,15054600,0.0,0.0
2025-01-10 00:00:00-05:00,423.003698,423.083378,413.430488,417.345459,20201100,0.0,0.0
...,...,...,...,...,...,...,...
2025-04-28 00:00:00-04:00,391.241729,392.020298,385.931501,390.443207,16579400,0.0,0.0
2025-04-29 00:00:00-04:00,390.582932,394.375987,389.664635,393.317932,14974000,0.0,0.0
2025-04-30 00:00:00-04:00,389.584773,395.933134,383.735526,394.535706,36461100,0.0,0.0


In [16]:
# 인덱스 초기화 
result = result.reset_index()
result

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2025-01-03 00:00:00-05:00,419.467282,422.405996,417.933202,421.728607,16662900,0.0,0.0
1,2025-01-06 00:00:00-05:00,426.360784,432.656586,423.850447,426.211365,20573600,0.0,0.0
2,2025-01-07 00:00:00-05:00,427.356962,429.000637,419.188356,420.752350,18139100,0.0,0.0
3,2025-01-08 00:00:00-05:00,421.838197,425.334764,419.925568,422.933990,15054600,0.0,0.0
4,2025-01-10 00:00:00-05:00,423.003698,423.083378,413.430488,417.345459,20201100,0.0,0.0
...,...,...,...,...,...,...,...,...
77,2025-04-28 00:00:00-04:00,391.241729,392.020298,385.931501,390.443207,16579400,0.0,0.0
78,2025-04-29 00:00:00-04:00,390.582932,394.375987,389.664635,393.317932,14974000,0.0,0.0
79,2025-04-30 00:00:00-04:00,389.584773,395.933134,383.735526,394.535706,36461100,0.0,0.0
80,2025-05-01 00:00:00-04:00,430.319981,436.189211,424.121369,424.620453,58938100,0.0,0.0


In [17]:
# 날짜 부분만 추출
result['Date'] = result['Date'].dt.strftime('%Y-%m-%d')

result

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2025-01-03,419.467282,422.405996,417.933202,421.728607,16662900,0.0,0.0
1,2025-01-06,426.360784,432.656586,423.850447,426.211365,20573600,0.0,0.0
2,2025-01-07,427.356962,429.000637,419.188356,420.752350,18139100,0.0,0.0
3,2025-01-08,421.838197,425.334764,419.925568,422.933990,15054600,0.0,0.0
4,2025-01-10,423.003698,423.083378,413.430488,417.345459,20201100,0.0,0.0
...,...,...,...,...,...,...,...,...
77,2025-04-28,391.241729,392.020298,385.931501,390.443207,16579400,0.0,0.0
78,2025-04-29,390.582932,394.375987,389.664635,393.317932,14974000,0.0,0.0
79,2025-04-30,389.584773,395.933134,383.735526,394.535706,36461100,0.0,0.0
80,2025-05-01,430.319981,436.189211,424.121369,424.620453,58938100,0.0,0.0


In [18]:
# 데이터프레임을 딕셔너리로 변환
result_dict = result.to_dict(orient='records') 
result_dict

[{'Date': '2025-01-03',
  'Open': 419.4672816518615,
  'High': 422.40599553232005,
  'Low': 417.93320162542307,
  'Close': 421.7286071777344,
  'Volume': 16662900,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Date': '2025-01-06',
  'Open': 426.3607841744266,
  'High': 432.6565862275622,
  'Low': 423.85044657634234,
  'Close': 426.21136474609375,
  'Volume': 20573600,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Date': '2025-01-07',
  'Open': 427.356962317073,
  'High': 429.00063686122985,
  'Low': 419.18835553907815,
  'Close': 420.7523498535156,
  'Volume': 18139100,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Date': '2025-01-08',
  'Open': 421.8381972488523,
  'High': 425.33476415419165,
  'Low': 419.9255676122581,
  'Close': 422.9339904785156,
  'Volume': 15054600,
  'Dividends': 0.0,
  'Stock Splits': 0.0},
 {'Date': '2025-01-10',
  'Open': 423.0036976315441,
  'High': 423.0833778600729,
  'Low': 413.4304875422897,
  'Close': 417.345458984375,
  'Volume': 20201100,
  'Divide

`(2) 데이터 연동 및 출력 포맷` 

In [19]:
# 외부 API 연동하는 함수 (yfinance 사용)

from langchain_core.tools import ToolException
from typing import Dict, Optional
from datetime import datetime, timedelta
import yfinance as yf

def get_stock_price(symbol: str, date: Optional[str] = None) -> Dict:
    """yfiance 사용하여 특정 날짜의 주식의 가격 정보를 조회합니다."""

    if date and not is_valid_date(date):
        raise ToolException(f"잘못된 날짜 형식입니다: {date}")
    
    try:
        stock = yf.Ticker(symbol)
        # 특정 날짜의 주식 가격 정보 조회 
        if date:
            start = datetime.strptime(date, "%Y-%m-%d")
            end = start + timedelta(days=1)
            price = stock.history(start=start, end=end)

            # 가격 정보가 없으면 해날 날짜로부터 과거 5일간의 주식 가격 정보 조회
            if price.empty:
                end = start - timedelta(days=5)
                price = stock.history(start=end, end=start)

        # 특정 날짜가 없으면 최근 5일간의 주식 가격 정보 조회
        else:
            price = stock.history(period="5d")
            
        # 데이터프레임을 딕셔너리로 변환하여 반환 (가장 최근 날짜 데이터만 반환)
        df = price.reset_index()
        df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')
        return df.to_dict(orient='records')[-1]
    
    except Exception as e:
        raise ToolException(str(e))
    

def is_valid_date(date_str: str) -> bool:
    try:
        datetime.strptime(date_str, '%Y-%m-%d')
        return True
    except ValueError:
        return False
    

# 함수 실행
result = get_stock_price("AAPL")

# 결과 출력
print(result)

{'Date': '2025-07-03', 'Open': 212.14999389648438, 'High': 214.64999389648438, 'Low': 211.80999755859375, 'Close': 213.5500030517578, 'Volume': 34955800, 'Dividends': 0.0, 'Stock Splits': 0.0}


In [20]:
# 함수 실행 (날짜 지정)
result = get_stock_price("AAPL", "2025-01-03")
print(result)

{'Date': '2025-01-03', 'Open': 242.7743682861328, 'High': 243.5923870862029, 'Low': 241.3079045417172, 'Close': 242.7743682861328, 'Volume': 40244100, 'Dividends': 0.0, 'Stock Splits': 0.0}


`(3) StructuredTool 도구 변환` 

In [21]:
from langchain_core.tools import StructuredTool

# StructuredTool로 도구 생성
stock_tool = StructuredTool.from_function(
    func=get_stock_price,
    name="stock_price_basic",
    description="yfinance를 사용하여 주식 가격 정보를 조회하는 도구입니다.",
)

# 도구 실행 (정상)
result = stock_tool.invoke({"symbol": "AAPL"})
print(result)

{'Date': '2025-07-03', 'Open': 212.14999389648438, 'High': 214.64999389648438, 'Low': 211.80999755859375, 'Close': 213.5500030517578, 'Volume': 34955800, 'Dividends': 0.0, 'Stock Splits': 0.0}


`(4) LLM 사용하여 도구 사용` 

In [22]:
from langchain_openai import ChatOpenAI

# OpenAI GPT-4o-mini 모델 사용
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 바인딩
llm_with_tool = llm.bind_tools([search_web, stock_tool])

# 도구 호출
result = llm_with_tool.invoke("애플의 주식 가격을 알려줘")
pprint(result.tool_calls)
print("-"*100)

# ToolCall을 도구에 전달하여 결과 확인
tool_call = result.tool_calls[0]
if tool_call['name'] == "stock_price_basic":
    print("도구 호출 이름:", tool_call['name'])
    result = stock_tool.invoke(tool_call)
elif tool_call['name'] == "tavily_search_results":
    print("도구 호출 이름:", tool_call['name'])
    result = search_web.invoke(tool_call)
else:
    raise ValueError(f"알 수 없는 도구 호출: {tool_call['name']}")

print(result)

[{'args': {'symbol': 'AAPL'},
  'id': 'call_RYVJKEagkEItG1QV4jliRuGU',
  'name': 'stock_price_basic',
  'type': 'tool_call'}]
----------------------------------------------------------------------------------------------------
도구 호출 이름: stock_price_basic
content='{"Date": "2025-07-03", "Open": 212.14999389648438, "High": 214.64999389648438, "Low": 211.80999755859375, "Close": 213.5500030517578, "Volume": 34955800, "Dividends": 0.0, "Stock Splits": 0.0}' name='stock_price_basic' tool_call_id='call_RYVJKEagkEItG1QV4jliRuGU'


In [23]:
# 도구 호출 (날짜 지정)
result = llm_with_tool.invoke("애플의 2025년 1월 3일 주식 가격을 알려줘")
pprint(result.tool_calls)

# ToolCall을 도구에 전달하여 결과 확인
for tool_call in result.tool_calls:
    print("도구 호출 이름:", tool_call['name'])
    if tool_call['name'] == "stock_price_basic":
        result = stock_tool.invoke(tool_call)
    elif tool_call['name'] == "tavily_search_results":
        result = search_web.invoke(tool_call)
    else:
        raise ValueError(f"알 수 없는 도구 호출: {tool_call['name']}")
    
    print(result)
    print("-" * 100)

[{'args': {'date': '2025-01-03', 'symbol': 'AAPL'},
  'id': 'call_c0756PjIP0SltC7JNZB5LeCT',
  'name': 'stock_price_basic',
  'type': 'tool_call'}]
도구 호출 이름: stock_price_basic
content='{"Date": "2025-01-03", "Open": 242.7743682861328, "High": 243.5923870862029, "Low": 241.3079045417172, "Close": 242.7743682861328, "Volume": 40244100, "Dividends": 0.0, "Stock Splits": 0.0}' name='stock_price_basic' tool_call_id='call_c0756PjIP0SltC7JNZB5LeCT'
----------------------------------------------------------------------------------------------------


In [24]:
from langgraph.prebuilt import create_react_agent

# 도구 실행 에인전트 생성 
stock_agent = create_react_agent(llm, tools=[search_web, stock_tool])

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 2025년 1월 3일 주식 가격을 알려줘")]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 2025년 1월 3일 주식 가격을 알려줘', additional_kwargs={}, response_metadata={}, id='8873a9bb-0f86-4c5d-a78f-16e5293d4b5f'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_uFwKS4ddTRJTcUl6UFuXDGPI', 'function': {'arguments': '{"symbol":"AAPL","date":"2025-01-03"}', 'name': 'stock_price_basic'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 140, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BpcyVLehOhkiUglTN4UMWvZg63Mwb', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--2e442c7a-eeb9-434f-aebc-f6c3a0261947-0', tool_calls=[{'name': 'stock_price_basic', 'args': {'symbol': '

In [25]:
# 도구 실행 에인전트 생성 (return_direct=True)

stock_tool.return_direct = True
search_web.return_direct = True
stock_agent = create_react_agent(llm, tools=[search_web, stock_tool])

# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 2025년 1월 3일 주식 가격을 알려줘")]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 2025년 1월 3일 주식 가격을 알려줘', additional_kwargs={}, response_metadata={}, id='aba0a60e-3da9-4b98-a657-6b8a3829070b'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2rnevhJaPiusZ8YtEF21W2Ox', 'function': {'arguments': '{"symbol":"AAPL","date":"2025-01-03"}', 'name': 'stock_price_basic'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 140, 'total_tokens': 165, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BpcycMelQtL1jqEPBMjKYW9UA4yQu', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--047629a5-36bc-4cef-9952-50094c959ff9-0', tool_calls=[{'name': 'stock_price_basic', 'args': {'symbol': '

In [26]:
# 도구 실행 에이전트 사용
result = stock_agent.invoke(
    {"messages": [("human", "애플의 최신 뉴스를 알려줘")]}
)

pprint(result['messages'])

[HumanMessage(content='애플의 최신 뉴스를 알려줘', additional_kwargs={}, response_metadata={}, id='8f76c930-149d-4ed1-9e6d-b8c15a9662be'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_PBG8jewgNWJ4VDSpOjweVJC4', 'function': {'arguments': '{"query":"Apple latest news"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 129, 'total_tokens': 149, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-BpcyhemF7yoOLj6RHNb1atU3DwGro', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ba55e7bd-8132-41a5-ae5b-99f2d64aa2d6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'App

---

### 2. **Naver 개발자 API** 를 도구로 사용

- 네이버 개발자 API(https://developers.naver.com/)에서 인증 권한 취득 (회원 가입 및 애플리케이션 등록 필요)
- 환경변수(.env)를 등록합니다. (**NAVER_CLIENT_ID**, **NAVER_CLIENT_SECRET**)
- 아래 정의된 네이버 뉴스 검색 도구(naver_news_search)를 사용하여 다음 과정을 수행합니다. 

In [27]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

True

`(1) 도구 정의`

In [28]:
import requests, os
from langchain_core.tools import tool
from typing import Dict

@tool
def naver_news_search(
    query: str,
    ) -> Dict[Dict, int]:
    """네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다."""

    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {"query": query}

    response = requests.get(url, headers=headers, params=params)

    return {
        "data": response.json(),
        "status_code": int(response.status_code)
    }  #type: ignore

`(2) 도구 실행`

In [29]:
from langchain_core.runnables import chain, RunnableLambda
from langchain_openai import ChatOpenAI

# 도구 맵 생성 (도구 이름을 키로 사용)
tools = [stock_tool, naver_news_search]  
tool_map = {tool.name: tool for tool in tools}

# 도구 라우터 (도구 이름에 따라 실행할 도구를 선택)
@chain
def tool_router(tool_call):
    return tool_map[tool_call["name"]]

# 체인 구성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

llm_with_tools = llm.bind_tools(tools)

tool_chain = (
    llm_with_tools    # 어떤 도구를 사용할지 결정 (LLM 모델이 도구 호출을 처리)
    | RunnableLambda(lambda x: x.tool_calls)  # 도구 호출을 추출
    | tool_router.map()   # 도구 호출 라우팅
)
# 도구 실행
response = tool_chain.invoke("애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")

In [30]:
# 결과 출력 (도구 호출 결과)
for msg in response:
    pprint(msg)

ToolMessage(content='{"Date": "2025-07-03", "Open": 212.14999389648438, "High": 214.64999389648438, "Low": 211.80999755859375, "Close": 213.5500030517578, "Volume": 34955800, "Dividends": 0.0, "Stock Splits": 0.0}', name='stock_price_basic', tool_call_id='call_nYw4aR5h3BBBL7Gws5EqvXNy')
ToolMessage(content='{"data": {"errorMessage": "NID AUTH Result Invalid (1000) : Authentication failed. (인증에 실패했습니다.)", "errorCode": "024"}, "status_code": 401}', name='naver_news_search', tool_call_id='call_n2TU88DjhsDXwhK05tlgDp6a')


`(3) 체인 실행`

In [32]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_core.messages import ToolMessage
from langchain_openai import ChatOpenAI
from datetime import datetime

# LLM 모델 인스턴스를 생성
llm = ChatOpenAI(model="gpt-4.1-mini")

# 두 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[stock_tool, naver_news_search])

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate([
    ("system", f"당신은 도움이 되는 AI 어시스턴트입니다. 오늘 날짜는 {today}입니다. 답변 생성의 근거 또는 출처를 명시하세요."),
    ("human", "{user_input}"),  
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def news_analysis_chain(user_input: str):

    # 도구 체인 실행 (LLM -> StockPrice/NaverNewsSearch)
    tool_msgs = tool_chain.invoke(user_input)

    print(f"Tool Messages: {tool_msgs}")

    # 도구 결과를 포맷팅
    formatted_messages = []
    for msg in tool_msgs:
        if isinstance(msg, ToolMessage) and len(msg.content) > 0:
            formatted_messages.append({"role": "assistant", "content": msg.content}) 

    print(f"Formatted Messages: {formatted_messages}")

    # 최종 응답 생성
    final_response = llm_chain.invoke({"user_input": user_input, "messages": formatted_messages})

    print(f"Final Response: {final_response}")

    return final_response

In [33]:
# 체인 실행
response = news_analysis_chain.invoke("애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")

Tool Messages: [ToolMessage(content='{"Date": "2025-07-03", "Open": 212.14999389648438, "High": 214.64999389648438, "Low": 211.80999755859375, "Close": 213.5500030517578, "Volume": 34955800, "Dividends": 0.0, "Stock Splits": 0.0}', name='stock_price_basic', tool_call_id='call_lJGUrZ2KRK4ibsoFx7TjUiPc'), ToolMessage(content='{"data": {"errorMessage": "NID AUTH Result Invalid (1000) : Authentication failed. (인증에 실패했습니다.)", "errorCode": "024"}, "status_code": 401}', name='naver_news_search', tool_call_id='call_UwsBuI0hA9HmfVhtY7jIBEUA')]
Formatted Messages: [{'role': 'assistant', 'content': '{"Date": "2025-07-03", "Open": 212.14999389648438, "High": 214.64999389648438, "Low": 211.80999755859375, "Close": 213.5500030517578, "Volume": 34955800, "Dividends": 0.0, "Stock Splits": 0.0}'}, {'role': 'assistant', 'content': '{"data": {"errorMessage": "NID AUTH Result Invalid (1000) : Authentication failed. (인증에 실패했습니다.)", "errorCode": "024"}, "status_code": 401}'}]
Final Response: content='' addi

In [ ]:
# 최종 응답 출력 
print(response.content)